In [ ]:
# pip install python-chess==0.31.3

# Creating the dataset

Now we need to convert the board representation to something meaningful.

A 3d matrix of sizes **14 x 8 x 8**



In [3]:
import chess
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

# AI vs AI

In [37]:
# used for the minimax algorithm
def minimax_eval(board, model):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model.predict(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player, model):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board, model)
  
  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, maximizing_player)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, maximizing_player)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# this is the actual function that gets the move from the neural network
def get_ai_move(board, depth, model, player):
  best_move = None
  max_eval = -numpy.inf
  min_eval = numpy.inf


  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, player, model)
    board.pop()
    if player:
      if eval > max_eval:
        max_eval = eval
        best_move = move
    else:
      if eval < min_eval:
        min_eval = eval
        best_move = move
  return best_move


In [38]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3,
                      padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)


In [39]:
import numpy
import tensorflow

model = tensorflow.keras.models.load_model('model_weight/model.h5')

# cen_model.load_weights(
#     "model_weight/centralized_weights")
# cen_model = build_model(32, 4)
# cen_model.load_weights(
#     "model_weight/centralized_weights")

# fed0_model = build_model(32, 4)
# fed0_model.load_weights(
#     "model_weight/federated0_weights")

# fed1_model = build_model(32, 4)
# fed1_model.load_weights(
#     "model_weight/federated1_weights")

# fed2_model = build_model(32, 4)
# fed2_model.load_weights(
#     "model_weight/federated2_weights")


In [40]:
# board = chess.Board()

# while True:
#     move = get_ai_move(board, 1, cen_model)
#     board.push(move)
#     print(f'\n{board}')
#     if board.is_game_over():
#         print('game_over')
#         break
#     move = get_ai_move(board, 1, fed2_model)
#     board.push(move)
#     print(f'\n{board}')
#     if board.is_game_over():
#         print('game_over')
#         print(board.outcome())
#         break


In [42]:
board = chess.Board()

while True:
    move = get_ai_move(board, 1, model, player=True)
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        break
    move = get_ai_move(board, 1, model, player=False)
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        print(board.outcome())
        break



r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R

r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R

r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . B .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N . Q K B N R

r n b . k b n r
p p p p . p p p
. . . . p . . .
. . . . . . q .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N . Q K B N R

r n b . k b n r
p p p p . p p p
. . . . p . . .
. . . . . . q .
. . . P . . . .
. . . . . N . .
P P P . P P P P
R N . Q K B . R

r n b . k . n r
p p p p . p p p
. . . . p . . .
. . . . . . q .
. b . P . . . .
. . . . . N . .
P P P . P P P P
R N . Q K B . R

r n b . k . n r
p p p p . p p p
. . . . p . . .
. . . . . . q .
. b . P . . . .
. . . . . N . .
P P P N P P P P
R . . Q K B . R

r n b . k . n r
p p p p . p p p
. . . . p . . .
. . . . . . q .
. . . P . . . .
. . . . . N . .
